In [1]:
import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pickle as p
import matplotlib.pyplot as plt
from scipy import *
import os
import h5py
from keras import backend as K
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, adam
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from sklearn.utils import shuffle
from keras.utils import np_utils
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, exposure
import pickle 
from skimage.transform import rescale, resize
import scipy.misc

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
# Setting directory paths 
root_dir = os.getcwd()

# Loading data
data = h5py.File(r'data.h5', 'r')

images_raw = np.asarray(data['images'])
labels = data['labels']

In [3]:
print(images_raw.shape)

(356, 267, 267)


In [4]:
# balance classes
def bal_classes(images, labels):
    class1 = []
    class2 = []
    
    for i in range(labels.shape[0]):
        if labels[i] == False:
            class1.append(i)
    
    for i in range(labels.shape[0]):
        if labels[i] == True:
            class2.append(i)
    
    if len(class2) > len(class1):
        class2 = np.asarray(class2[:len(class1)])
    
    elif len(class2) < len(class1):
        class1 = np.asarray(class1[:len(class2)])
        
    images1 = images[class1, :] 
    images2 = images[class2, :]
    labels1 = labels[class1,]
    labels2 = labels[class2,]
    
    print (labels2.shape)
    
    images_st = np.vstack((images1, images2))
    labels_st =  np.concatenate((labels1, labels2), axis=0)              
        
    images_s, labels_s = shuffle(np.asarray(images_st), np.asarray(labels_st), random_state = 0)    
        
    return images_s, labels_s

In [5]:
images_p, labels_p = bal_classes(np.asarray(images_raw), np.asarray(labels))

(123,)


In [6]:
# Defining rescaling function
def rescale_imgs(images):
    count =  0
    for i in range(0 ,images.shape[0]):
        img_scaled = expand_dims(resize(images[i, :, :], (256, 256)), axis = 0)
        if count == 0:
            imgs = img_scaled
        else:
            imgs = np.vstack((imgs, img_scaled))
        count += 1
                             
    return imgs

In [7]:
images_shrunk = rescale_imgs(images_p)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [8]:
print (images_shrunk.shape)

(246, 256, 256)


In [9]:
def format_data(images, labels, ratio = 0.5):
    split = round(ratio*images.shape[0])
    
    train_x = images[ :split, :]
    test_x = images[split:, :]
    train_y = labels[ :split]
    test_y = labels[split:]
    
    return train_x, test_x, train_y, test_y


train_x, test_x, train_y, test_y = format_data(images_shrunk, labels_p)

In [10]:
train_x.shape

(123, 256, 256)

In [11]:
#imgs = np.reshape(images_shrunk, (-1, images_shrunk.shape[1], images_shrunk.shape[2]))

#plt.figure(figsize=(12, 12))
#for n in range(0, 16):
    #plt.subplot(4, 4, 1 + n)
    #plt.imshow(stretch(imgs[-n-1,:,:]), cmap='viridis', shape=(135, 135))
    #plt.axis('off')
    #plt.title(labels[n])

In [12]:
#stretch = vis.AsinhStretch(1e-1) + vis.MinMaxInterval()

#images_s = np.zeros((images_shrunk.shape[0], images_shrunk.shape[1], images_shrunk.shape[2]))

#for i, images in enumerate(images_shrunk):
    #images_s[i, :, :] = stretch(images_shrunk[i, :, :])

In [13]:
def data_gen(images, labels):

    batch_size = 20
    samples = np.asarray(images).shape[0]

    datagen = ImageDataGenerator(
            rotation_range=180,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip =True,
            fill_mode = "nearest")

    x = images.reshape((-1, images.shape[1],images.shape[2], 1))

    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    gen = datagen.flow(x, np.asarray(labels), batch_size=batch_size)

    data_mult = 100

    pro_images = np.zeros((images.shape[0]*data_mult, images.shape[1], images.shape[2], 1))
    pro_labels = np.zeros((labels.shape[0]*data_mult))
    
    for e in range(1, data_mult+1):
        batch = 1
        b = batch_size
        b_start = samples*(e-1)

        for X_batch, Y_batch in gen:
            if batch < (samples/b):
                pro_images[b_start+b*(batch-1):b_start+batch*b, :, :, :] = X_batch
                pro_labels[b_start+b*(batch-1):b_start+batch*b] = Y_batch

            else: 
                pro_images[b_start+b*(batch-1):b_start+b*(batch-1) + X_batch.shape[0]%b, :, :, :] = X_batch
                pro_labels[b_start+b*(batch-1):b_start+b*(batch-1) + X_batch.shape[0]%b] = Y_batch
                break


            batch += 1
        print(e)
        
    return pro_images, pro_labels

In [14]:
#pro_train_x, pro_train_y = data_gen(train_x, train_y)
#pro_test_x, pro_test_y  = data_gen(test_x, test_y)

In [15]:
#pro_train_x = np.reshape(pro_train_x, (pro_train_x.shape[0], pro_train_x.shape[1], pro_train_x.shape[2]))
#pro_test_x = np.reshape(pro_test_x, (pro_test_x.shape[0], pro_test_x.shape[1], pro_test_x.shape[2]))

NameError: name 'pro_train_x' is not defined

In [ ]:
#imgs = np.reshape(pro_images, (-1, pro_images.shape[1], pro_images.shape[2]))

#for n in range(0, imgs.shape[0]):
   #plt.imshow(imgs[n,:,:], cmap='gray', shape=(89, 89))
   #print (n)
   #print (pro_labels[n])
   #plt.show()

In [16]:
with open('train_x.dat', 'wb') as f:
    pickle.dump(train_x, f, protocol=-1)

In [17]:
with open('test_x.dat', 'wb') as f:
    pickle.dump(test_x, f, protocol=-1)

In [18]:
with open('train_y.dat', 'wb') as f:
    pickle.dump(train_y, f, protocol=-1)    

with open('test_y.dat', 'wb') as f:
    pickle.dump(test_y, f, protocol=-1)

In [19]:
print ("Done")

Done
